# Chains
LangChain is primarily based on chains that combine the prompt template, llm and the output format in a single chain/component.

You do not need to define the prompt template, the llm model, the output format, you just use LLMChain which handles the rest.

In [21]:
!pip install langchain
!pip install langchain_community

# For Open-AI
!pip install langchain-core
!pip install langchain_openai

# For Google Generative AI
!pip install langchain-google-genai
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.9 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.6 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [14]:
import os
os.environ["OPENAI_API_KEY"] =  "your-api-key"
os.environ["GOOGLE_API_KEY"] =  "your-api-key"

## GenericChain - LLMChain

In [5]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

dictionary_template = PromptTemplate(
    input_variables=["word"],
    template="Give mw one line definition of {word}. Then, give one example of how it is used."
)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# output_parser is an optional parameter.
chain = LLMChain(llm=llm, prompt=dictionary_template)
chain.run("Anachronism") # can also use chain.invoke()

'Anachronism is something that is out of its proper time period.\n\nExample: In the movie "Gladiator," the character Maximus wears a wristwatch, which is an anachronism because wristwatches were not invented until centuries after the time period in which the movie is set.'

In [6]:
# .invoke() is the latest method
chain.invoke("Anachronism")

{'word': 'Anachronism',
 'text': 'Anachronism is something that is out of its proper time period.\n\nExample: In the movie "Gladiator," the character Maximus wears a wristwatch, which is an anachronism because wristwatches were not invented until centuries after the time period in which the movie is set.'}

## Batch Running

You can give a batch of inputs to get a batch of responses

In [7]:
input_list =[
    {"word": "Humor"},
    {"word":"Politician"},
    {"word":"Anachronism"}
]

# chain is already given the llm and prompt template

chain.apply(input_list)

[{'text': "Humor is the quality of being amusing or entertaining, especially through the use of wit or cleverness.\n\nExample: The comedian's stand-up routine was filled with clever jokes and witty observations that had the audience laughing uncontrollably."},
 {'text': 'Politician: A person who is professionally involved in politics, especially as a holder of an elected office.\n\nExample: The politician promised to lower taxes and increase funding for education during his campaign.'},
 {'text': 'Anachronism is something that is out of its proper time period.\n\nExample: In the movie "Gladiator," the character Maximus wears a wristwatch, which is an anachronism because wristwatches were not invented until centuries after the time period in which the movie is set.'}]

In [8]:
"""
  Generate other information along with ouput like
  which model used and other metadata
"""

chain.generate(input_list)

LLMResult(generations=[[ChatGeneration(text="Humor is the quality of being amusing or entertaining, especially through the use of jokes or comedy.\n\nExample: The comedian's witty one-liners had the audience roaring with laughter throughout the entire show.", generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content="Humor is the quality of being amusing or entertaining, especially through the use of jokes or comedy.\n\nExample: The comedian's witty one-liners had the audience roaring with laughter throughout the entire show.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 27, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BqGQVVCNCECEu

# Utility Chains

Lang chain docs do not have a terminology for it, but we are calling the below as utility chains because each serve a specific purpose.

### **MathChain**

Does not send the prompt but ask the llm for python code for the problem and then run the python code to return the result.

#### Why not give prompt directly ?.

Because the LLM does not solve rather try to guess the value and oftentimes when the calculation is very complex it gets wrong answer guess. So `LLMMathChain` solves it rather answer get python code and run to get perfect ansswer.

In [9]:
from langchain.chains import LLMMathChain

In [10]:
calculator = LLMMathChain.from_llm(llm=llm, verbose=True)
calculator.invoke("Calculate(6 raised to power (0.225)) - (log10(100))")



> Entering new LLMMathChain chain...
Calculate(6 raised to power (0.225)) - (log10(100))```text
6**0.225 - log10(100)
```
...numexpr.evaluate("6**0.225 - log10(100)")...

Answer: -0.5034748103023869
> Finished chain.


{'question': 'Calculate(6 raised to power (0.225)) - (log10(100))',
 'answer': 'Answer: -0.5034748103023869'}

In [11]:
# see what prompt did the chain sent to llm
print(calculator.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



#### Without LLMMathChain

You can compare the answer, which is not correct with the one abve calculated using Chain.

In [12]:
calc_llm = ChatOpenAI()
messages = [
    ("human", "Calculate(6 raised to power (0.225)) - (log10(100))")
]
calculation = calc_llm.invoke(messages)
print(calculation.content)

6^0.225 ≈ 1.8461
log10(100) = 2

Therefore, the calculation is:

1.8461 - 2 = -0.1539


# Sequential Chain

In [18]:
marketing_text = PromptTemplate(
    input_variables = ["name", "description"],
    template = "Generate a one line Facebook ad copy for a product called {name}. \
                 Below is the description of the product which is {description}"
)

text_llm = ChatOpenAI()

text_chain = LLMChain(llm=text_llm, prompt=marketing_text, output_key="copy_text")

translate_text = PromptTemplate(input_variables=["copy_text"], template="Translate this to Punjabi(pakistani urdu style) text: {copy_text}")

from langchain_google_genai import ChatGoogleGenerativeAI

translate_llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-pro")

translate_chain = LLMChain(llm=translate_llm, prompt=translate_text, output_key="translated_copy")

from langchain.chains import SequentialChain

seq_chain = SequentialChain(
    chains=[text_chain, translate_chain],
    input_variables=["name", "description"],
    output_variables=["copy_text", "translated_copy"]
)


seq_chain(
    {
        "name": "AeroGlow Nightlight",
        "description":"AeroGlow Nightlight is a smart, voice-activated nightlight that projects calming\
                       animated constellations onto your ceiling."
    }
)

{'name': 'AeroGlow Nightlight',
 'description': 'AeroGlow Nightlight is a smart, voice-activated nightlight that projects calming                       animated constellations onto your ceiling.',
 'copy_text': 'Transform your room into a serene starry night sky with AeroGlow Nightlight - the voice-activated smart nightlight. 🌌✨',
 'translated_copy': 'اپنے کمرے نوں ایروگلو نائٹ لائٹ نال اک پرسکون تاروں بھری رات دے آسمان وچ بدلو - آواز نال چلن والی سمارٹ نائٹ لائٹ۔ 🌌✨'}